In [1]:
# Import Libraries
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import pickle
from scipy.stats import mannwhitneyu

In [2]:
# Load the datasets
control_df = pd.read_csv("../data/raw/control_group.csv",sep = ";")

control_df.head(5)

,Campaign Name,Date,Spend [USD],# of Impressions,Reach,# of Website Clicks,# of Searches,# of View Content,# of Add to Cart,# of Purchase
0,Control Campaign,1.08.2019,2280,82702.0,56930.0,7016.0,2290.0,2159.0,1819.0,618.0
1,Control Campaign,2.08.2019,1757,121040.0,102513.0,8110.0,2033.0,1841.0,1219.0,511.0
2,Control Campaign,3.08.2019,2343,131711.0,110862.0,6508.0,1737.0,1549.0,1134.0,372.0
3,Control Campaign,4.08.2019,1940,72878.0,61235.0,3065.0,1042.0,982.0,1183.0,340.0
4,Control Campaign,5.08.2019,1835,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [3]:
print(control_df.shape)

(30, 10)


In [4]:
test_df = pd.read_csv("../data/raw/test_group.csv",sep = ";")

test_df.head(5)

,Campaign Name,Date,Spend [USD],# of Impressions,Reach,# of Website Clicks,# of Searches,# of View Content,# of Add to Cart,# of Purchase
0,Test Campaign,1.08.2019,3008,39550,35820,3038,1946,1069,894,255
1,Test Campaign,2.08.2019,2542,100719,91236,4657,2359,1548,879,677
2,Test Campaign,3.08.2019,2365,70263,45198,7885,2572,2367,1268,578
3,Test Campaign,4.08.2019,2710,78451,25937,4216,2216,1437,566,340
4,Test Campaign,5.08.2019,2297,114295,95138,5863,2106,858,956,768


In [5]:
print(test_df.shape)

(30, 10)


In [6]:
# Null Values

def remove_null_values(df):
    # Print the count of null values in each column
    print("Null values count in each column:")
    null_counts = df.isnull().sum()
    print(null_counts)

    if null_counts.sum() == 0:
        print("DataFrame has no null values")
    else:
        # Remove rows with null values and update the DataFrame
        df = df.dropna()
        print("Shape after removing null values:")
        print(df.shape)
    
    return df

In [7]:
# Duplicate Values
def check_and_remove_duplicates(df):
    
    duplicate_count = df.duplicated().sum()
    
    if duplicate_count > 0:
        
        print(f"Number of duplicate rows: {duplicate_count}")

        
        df = df.drop_duplicates()
        
        print("Duplicate rows removed.")
    else:
        print("No duplicate rows found.")

In [8]:
def reset_dataframe_index(df):
    # Reset the index and drop the old index
    df = df.reset_index(drop=True)
    
reset_dataframe_index(control_df)
reset_dataframe_index(test_df)

In [9]:
def calculate_and_remove_outliers(df, iqr_threshold=1.5):
    # Specify the columns you want to analyze for outliers
    columns_to_analyze = ['Spend [USD]', '# of Impressions', 'Reach',
                          '# of Website Clicks', '# of Searches', '# of View Content',
                          '# of Add to Cart', '# of Purchase']


    Q1 = df[columns_to_analyze].quantile(0.25)
    Q3 = df[columns_to_analyze].quantile(0.75)

    # Calculate the IQR
    IQR = Q3 - Q1

    # Determine lower and upper bounds for outliers
    lower_bound = Q1 - iqr_threshold * IQR
    upper_bound = Q3 + iqr_threshold * IQR

    # Identify outliers for the specified columns in each row
    outliers = ((df[columns_to_analyze] < lower_bound) | (df[columns_to_analyze] > upper_bound)).any(axis=1)

    # Calculate the percentage of outliers
    outlier_percentage = (outliers.sum() / len(control_df)) * 100

    # Remove rows with outliers
    df = df[~outliers]

    print(f"Removed {outlier_percentage:.2f}% of outliers from the DataFrame.")
  

In [10]:
# Assign a 'name' attribute to your DataFrames to use in the function calls
control_df.name = 'control_df'
test_df.name = 'test_df'

# Pass DataFrames through the functions
control_df = remove_null_values(control_df)
check_and_remove_duplicates(control_df)
reset_dataframe_index(control_df)
calculate_and_remove_outliers(control_df)
print("\n")
test_df = remove_null_values(test_df)
check_and_remove_duplicates(test_df)
reset_dataframe_index(test_df)
calculate_and_remove_outliers(test_df)

Null values count in each column:
Campaign Name          0
Date                   0
Spend [USD]            0
# of Impressions       1
Reach                  1
# of Website Clicks    1
# of Searches          1
# of View Content      1
# of Add to Cart       1
# of Purchase          1
dtype: int64
Shape after removing null values:
(29, 10)
No duplicate rows found.
Removed 3.45% of outliers from the DataFrame.


Null values count in each column:
Campaign Name          0
Date                   0
Spend [USD]            0
# of Impressions       0
Reach                  0
# of Website Clicks    0
# of Searches          0
# of View Content      0
# of Add to Cart       0
# of Purchase          0
dtype: int64
DataFrame has no null values
No duplicate rows found.
Removed 0.00% of outliers from the DataFrame.


In [11]:
# Convert the "Date" column to datetime if it's in string format
control_df['Date'] = pd.to_datetime(control_df['Date'], format='%d.%m.%Y')
test_df['Date'] = pd.to_datetime(test_df['Date'], format='%d.%m.%Y')

# Sort the DataFrames by date
control_df = control_df.sort_values(by='Date')
test_df = test_df.sort_values(by='Date')

In [12]:
# upload to to processed
control_df.to_pickle("../data/interim/01_control_df_processed.pkl")
test_df.to_pickle("../data/interim/01_test_df_processed.pkl")